In [1]:
import time
import os
import sys
import logging

import numpy as np
import tenseal as ts

from sklearn.model_selection import train_test_split



# definition of parameters

In [3]:
# data_path = "./data/raw"            # whole data set
DATA_PATH = "./data/quick_demo/"   # subset of the data set, with 15 train_samples and 5 test_samples
LOG_PATH = "."
LOG_FILENAME = "date+all"

In [2]:
fileHandler = logging.FileHandler("{0}/{1}.log".format(LOG_PATH, LOG_FILENAME))
streamHandler = logging.StreamHandler(sys.stdout)
logging.basicConfig(format="%(asctime)s  [%(levelname)-8.8s]  %(message)s", datefmt='%m/%d/%Y %I:%M:%S %p', level = logging.DEBUG, handlers=[fileHandler, streamHandler])

07/15/2020 09:35:28 AM  [INFO    ]  This message should appear on the console


In [4]:
def crytp_array(X, local_context):
    res = []
    for vec in X:
        res.append(ts.ckks_vector(local_context, vec))
    return res

# Loading and processing the data

In [5]:
raw_train, raw_test = preprocessing.data_import(data_path)
train, submission_test = preprocessing.preprocessing(raw_train, raw_test)

NameError: name 'preprocessing' is not defined

In [ ]:
train, test = train_test_split(train, test_size=0.15)
train_labels = train.Survived
test_labels = test.Survived
train_features = train.drop("Survived", axis=1)
test_features = test.drop("Survived", axis=1)

# Definition of safety parameters

In [10]:
logging.info('Definition of safety parameters...')
timer = time.time()
# context = ts.context(ts.SCHEME_TYPE.CKKS, 32768,
# coeff_mod_bit_sizes=[60, 40, 40, 40, 40, 40, 40, 40, 40, 40, 40, 40, 40, 40, 40, 60])
# context = ts.context(ts.SCHEME_TYPE.CKKS, 8192, coeff_mod_bit_sizes=[40, 21, 21, 21, 21, 21, 21, 40])

context = ts.context(ts.SCHEME_TYPE.CKKS, 16384, coeff_mod_bit_sizes=[60, 40, 40, 40, 40, 40, 40, 40, 60])
context.global_scale = pow(2, 40)
logging.info("Done. " + str(round(time.time() - timer, 2)) + " seconds")
logging.info('Generation of the Galois Key...')
timer = time.time()
context.generate_galois_keys()
logging.info("Done. " + str(round(time.time() - timer, 2)) + " seconds")

07/15/2020 09:42:54 AM  [INFO    ]  Definition of safety parameters...
07/15/2020 09:42:54 AM  [INFO    ]  Done. 0.43 seconds
07/15/2020 09:42:54 AM  [INFO    ]  Generation of Galois Key...
07/15/2020 09:42:59 AM  [INFO    ]  Done. 4.65 seconds


# Data encryption

In [11]:
logging.info("Data encryption...")
timer = time.time()
encrypted_X = crytp_array(train_features.to_numpy(), context)
encrypted_Y = crytp_array(train_labels.to_numpy().reshape((-1, 1)), context)
encrypted_test_X = crytp_array(test_features.to_numpy(), context)
encrypted_test_Y = crytp_array(test_labels.to_numpy().reshape((-1, 1)), context)
logging.info("Done. " + str(round(time.time() - timer, 2)) + " seconds")

07/15/2020 09:43:49 AM  [INFO    ]  Data encryption...


NameError: name 'train_features' is not defined

# Training

In [12]:
logging.info("Training starting")
model = LogisticRegressionHE(local_context=context, safety=False, verbose=True, num_iter=40)
model.fit(encrypted_X, encrypted_Y, X_ne=train_features.to_numpy(), Y_ne=train_labels.to_numpy().reshape((-1, 1)))
logging.info("Training done. " + str(round(time.time() - timer, 2)) + " seconds")

07/15/2020 09:49:07 AM  [INFO    ]  Training starting


NameError: name 'LogisticRegressionHE' is not defined

# Accuracy

In [ ]:
acc = model.encrypted_accuracy(encrypted_test_X, encrypted_test_Y)
logging.info("Accuracy : %s " % acc[0])